### XXXXXX

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')

In [2]:
from utils.returns_data_class import ReturnsData
PERIOD = 1
data = ReturnsData(
    daily_returns_path="../Data/returns_df_611.csv",
    extras_path="../Data/historical_stocks.csv",
)
data.change_returns_period(PERIOD)
X = data.returns_df.values.T

No change made because period entered is 1


In [3]:
X.shape

(611, 4692)

In [4]:
num_TS = X.shape[0]
num_pos_samples = 20
period = 20
stride = 3
num_neg_samples = 20

print(f"Context Size: {num_pos_samples}, Period: {period}, Stride: {stride}")
print(f"Number of Negative Samples: {num_neg_samples}")

from utils.context import get_tgt_context_euclidean_multiprocess
positive_tgt_context_sets = get_tgt_context_euclidean_multiprocess(ts_array=X, m=period, k=num_pos_samples, stride=stride, z_normalize=False, verbose=True)
negative_tgt_context_sets = get_tgt_context_euclidean_multiprocess(ts_array=X, m=period, k=num_neg_samples, stride=stride, z_normalize=False, top_k=False, verbose=True)
print(f"Number (anchor, positive, negative) samples: {len(positive_tgt_context_sets)}")

Context Size: 20, Period: 20, Stride: 3
Number of Negative Samples: 20


100%|██████████| 87/87 [00:11<00:00,  7.76it/s]


nearly returning


100%|██████████| 87/87 [00:10<00:00,  8.45it/s]


nearly returning
Number (anchor, positive, negative) samples: 951938


In [5]:
index_samples = []
for pos, neg in zip(positive_tgt_context_sets,negative_tgt_context_sets):
    index_samples.append((pos[0], pos[1], neg[1]))

In [118]:
import torch
from torch.utils.data import Dataset, DataLoader


class MultiPosNegDataset(Dataset):
    def __init__(self, index_samples):
        self.index_samples = index_samples

    def __len__(self):
        return len(self.index_samples)

    def __getitem__(self, idx):
        anchor_idx, positive_indices, negative_indices = self.index_samples[idx]
        # Convert indices list to tensor

        positive_indices_tensor = torch.tensor(positive_indices)
        negative_indices_tensor = torch.tensor(negative_indices)
        return anchor_idx, positive_indices_tensor, negative_indices_tensor


def normalize_embeddings(embeddings):
    with torch.no_grad():  # Ensure no gradients are computed in this block
        norms = embeddings.weight.norm(dim=1, keepdim=True)
        embeddings.weight.data = embeddings.weight.data / norms
    return embeddings


# def experimental_loss(
#     anchor_embeddings: torch.Tensor,
#     positive_embeddings: torch.Tensor,
#     negative_embeddings: torch.Tensor,
# ) -> torch.Tensor:
#     """

#     Args:
#         anchor_embeddings (torch.Tensor): shape (batch_size, embedding_dim)
#         positive_embeddings (torch.Tensor): (batch_size, num_pos_samples, embedding_dim)
#         negative_embeddings (torch.Tensor): (batch_size, num_neg_samples, embedding_dim)

#     Returns:
#         torch.Tensor: _description_
#     """
#     positive_scores = torch.einsum(
#             "bpd,bd->bp", [positive_embeddings, anchor_embeddings]
#         )
#     negative_scores = torch.einsum(
#             "bnd,bd->bn", [negative_embeddings, anchor_embeddings]
#         )
#     # positive_loss = - torch.sum(torch.nn.functional.logsigmoid(positive_scores), dim=1)
#     # negative_loss = - torch.sum(torch.log(1-torch.sigmoid(negative_scores)), dim=1)
#     criterion = torch.nn.BCEWithLogitsLoss()
#     positive_loss = criterion(positive_scores, torch.ones_like(positive_scores))
#     negative_loss = criterion(negative_scores, torch.zeros_like(negative_scores))
#     loss = torch.sum(positive_loss + negative_loss)

#     return loss

def experimental_loss(
    anchor_embeddings: torch.Tensor,
    positive_embeddings: torch.Tensor,
    negative_embeddings: torch.Tensor,
) -> torch.Tensor:
    """

    Args:
        anchor_embeddings (torch.Tensor): shape (batch_size, embedding_dim)
        positive_embeddings (torch.Tensor): (batch_size, num_pos_samples, embedding_dim)
        negative_embeddings (torch.Tensor): (batch_size, num_neg_samples, embedding_dim)

    Returns:
        torch.Tensor: _description_
    """
    positive_scores = torch.einsum(
            "bd,bd->b", [torch.mean(positive_embeddings, dim=1), anchor_embeddings]
        )
    negative_scores = torch.einsum(
            "bd,bd->b", [torch.mean(negative_embeddings, dim=1), anchor_embeddings]
        )
    # positive_loss = - torch.sum(torch.nn.functional.logsigmoid(positive_scores))
    # negative_loss = - torch.sum(torch.log(1-torch.sigmoid(negative_scores)+0.00001))
    criterion = torch.nn.BCEWithLogitsLoss()
    positive_loss = criterion(positive_scores, torch.ones_like(positive_scores))
    negative_loss = criterion(negative_scores, torch.zeros_like(negative_scores))

    loss = positive_loss + negative_loss

    return loss

def experimental_loss(
    anchor_embeddings: torch.Tensor,
    positive_embeddings: torch.Tensor,
    negative_embeddings: torch.Tensor,
) -> torch.Tensor:
    """

    Args:
        anchor_embeddings (torch.Tensor): shape (batch_size, embedding_dim)
        positive_embeddings (torch.Tensor): (batch_size, num_pos_samples, embedding_dim)
        negative_embeddings (torch.Tensor): (batch_size, num_neg_samples, embedding_dim)

    Returns:
        torch.Tensor: _description_
    """
    ### POSITIVE LOSS - Sigmoid over each positive
    positive_scores = torch.einsum(
            "bpd,bd->bp", [positive_embeddings, anchor_embeddings]
        )
    criterion = torch.nn.BCEWithLogitsLoss()
    positive_loss = criterion(positive_scores, torch.ones_like(positive_scores))

    ### NEGATIVE LOSS - Softmax with aggregate positive representation
    criterion = torch.nn.NLLLoss()
    aggregate_positive_embeddings = torch.mean(positive_embeddings, dim=1) # (batch_size, embedding_dim)
    aggregate_positive_scores = torch.einsum(
            "bd,bd->b", [aggregate_positive_embeddings, anchor_embeddings]
        )
    # aggregate_positive_scores = torch.mean(positive_scores, dim=1)
    negative_scores = torch.einsum(
            "bnd,bd->bn", [negative_embeddings, anchor_embeddings]
        )
    #-- Make first column the aggregate positive scores
    #- concatenated.shape = (batch_size, 1+num_neg_samples)
    concatenated = torch.concat((aggregate_positive_scores.unsqueeze(1), negative_scores), dim=1)
    #-- Create target for NLLLoss
    target = torch.zeros(concatenated.shape[0], dtype=torch.long)

    negative_loss = criterion(torch.nn.functional.log_softmax(concatenated, dim=1), target)
    loss = positive_loss + negative_loss

    return loss

In [128]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Parameters
num_items = num_TS  # Number of items in your dataset
embedding_size = 16  # Size of each embedding
learning_rate = 0.01
epochs = 2
batch_size = 2048

# Initialize the embedding matrix
embeddings = nn.Embedding(num_embeddings=num_items, embedding_dim=embedding_size)

# Optimizer
optimizer = optim.Adam(embeddings.parameters(), lr=learning_rate)

# Prepare your index_samples as a tensor
# index_samples = torch.tensor([(35, 12, 98), (47, 12, 4), ...])  # Your index tuples as a tensor
# Create the dataset and data loader
npair_dataset = MultiPosNegDataset(index_samples)
data_loader = DataLoader(npair_dataset, batch_size=batch_size, shuffle=True)

losses = []
REGULARIZATION_WEIGHT = 1
# Training loop with batching
for epoch in range(epochs):
    total_loss = 0
    # normalise the embeddings to prevent degenerate solution
    embeddings = normalize_embeddings(embeddings)

    for i, (anchor_idx, positive_idx, negative_indices) in enumerate(data_loader):

        # Get the embeddings for anchor, positive, and negative
        anchor_embeddings = embeddings(anchor_idx) # shape: (batch_size, embedding_dim)
        positive_embeddings = embeddings(positive_idx) # shape: (batch_size, num_pos_samples embedding_dim)
        negative_embeddings = embeddings(negative_indices) # shape: (batch_size, num_neg_samples, embedding_dim)

        # Compute the loss
        loss = experimental_loss(anchor_embeddings, positive_embeddings, negative_embeddings)
        regularization_loss = REGULARIZATION_WEIGHT * torch.square(embeddings.weight.norm(dim=1) - 1).sum()
        loss = loss + regularization_loss
        total_loss += loss.item()
        losses.append((loss.item(), regularization_loss.item()))

        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch % 10 == 0) | (epoch==epochs-1) | (epoch==0):
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {total_loss/len(data_loader)}')

embeddings = normalize_embeddings(embeddings)

Epoch [1/2], Loss: 3.1179292017413722
Epoch [2/2], Loss: 2.87751510835463


In [129]:
import plotly.express as px
losses=np.array(losses)
px.scatter(losses[:,0])

In [130]:
px.scatter(losses[:,1])

In [88]:
experimental_loss(anchor_embeddings, positive_embeddings, negative_embeddings)


tensor(1.6216, grad_fn=<NllLossBackward0>)

In [47]:
torch.concat((temp, temp), dim=1).shape

torch.Size([1666, 40])

### Get training data

In [131]:
from utils.sector_classification import get_sector_score

get_sector_score(embeddings.weight.detach().numpy(), sectors=data.sectors, top_k_accuracy=True)

/opt/homebrew/Caskroom/miniforge/base/envs/phd/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



Precision Score: 0.55
Recall Score: 0.4
F1 Score: 0.4
Accuracy Score: 0.4
Accuracy Score Top-3: 0.76


In [18]:
from utils.visualisation_functions import pca_plot_from_embeddings

pca_plot_from_embeddings(
    embedding_matrix=embeddings.weight.detach().numpy(),
    sectors=data.sectors,
    tickers=data.tickers,
    industries=data.industries,
    names=data.names,
    dimensions=2,
    reduced=True,
    method="PCA",
    return_df=False,
    rand_state=None,
)